In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib as mpl
import pandas as pd
import numpy as np

In [15]:
#Problem 1:
#First, enter the givens
fan_A = [55,52,51,59,60,56,54,54]
fan_B = [46,55,59,50,47,62,53,55]
alpha = 0.05
n = len(fan_A)
df1 = pd.DataFrame()
df1['fan_A'] = fan_A
df1['fan_B'] = fan_B
df1

,fan_A,fan_B
0,55,46
1,52,55
2,51,59
3,59,50
4,60,47
5,56,62
6,54,53
7,54,55


In [22]:
#To establish ranks, we create separate columns of df where the rank placement is considered
alldata = fan_A + fan_B
sorted(alldata)
#The labelling of rankings is done by hand and then transferred to Python

[46, 47, 50, 51, 52, 53, 54, 54, 55, 55, 55, 56, 59, 59, 60, 62]

In [23]:
df1['rank_A'] = [10,5,4,13.5,15,12,7.5,7.5] #Associated with Fan A ranks
df1['rank_B'] = [1,10,13.5,3,2,16,6,10] #Associated with Fan B ranks
df1

,fan_A,fan_B,rank_A,rank_B
0,55,46,10.0,1.0
1,52,55,5.0,10.0
2,51,59,4.0,13.5
3,59,50,13.5,3.0
4,60,47,15.0,2.0
5,56,62,12.0,16.0
6,54,53,7.5,6.0
7,54,55,7.5,10.0


In [27]:
#Defining key stuff for Wilcoxon, including sum totals of ranks:
n1 = n
n2 = n
T_A = df1['rank_A'].sum()
T_B = df1['rank_B'].sum()
T_A,T_B

(74.5, 61.5)

In [28]:
#Now that we have the calculated values, we compare to the table A.11 values for this stuation
#To identify if one of the fans is better, we compare the rank values to the 0.05 one-tailed bounds:
T_Lower = 52
T_Upper = 84
#Given that the values of T_A and T_B are within the lower and upper bounds, neither of the one sided hypothesis tests
#differentiating the two fans let us reject the null hypothesis when alpha = 0.05. Thus, the conclusion is that 
#we are unable to prove in a statistically significant way which fan is better. 
#However, looking by raw values, if a decision had to be made, it would be said that Fan A is better than Fan B
#based on a higher rank sum.

In [76]:
#Problem 2:
#First, we define the data:
Loc_A = [50,45,57]
Loc_B = [80,52,74]
Loc_C = [62,48,68]
df = pd.DataFrame()
df['Loc_A'] = Loc_A
df['Loc_B'] = Loc_B
df['Loc_C'] = Loc_C
df

,Loc_A,Loc_B,Loc_C
0,50,80,62
1,45,52,48
2,57,74,68


In [77]:
#Part A:
#Defining basic values relevant for the calculation:
alpha = 0.05
dof1 = df.shape[1] - 1
dof2 = df.shape[0]*df.shape[1] - df.shape[1]
dof1,dof2

(2, 6)

In [78]:
#Now, must calculate SSTr, SSE, and SST:
from scipy.stats import f
F_crit = f.ppf(0.95,dof1,dof2)
F_crit #This is compared against the F_0 value to reject or accept H_o

5.143252849784718

In [81]:
#Key calculations of all of the sums of squares
df_mean = df.mean()
df_std = df.std()
overall_mean = df.stack().mean()
n_i= df.count()
SSTr = (n_i*(df_mean-overall_mean)**2).sum()
SSTr

486.22222222222257

In [108]:
SSE = ((n_i-1)*df_std**2).sum()
SSE

718.0000000000001

In [83]:
SST = SSTr + SSE
SST

1204.2222222222226

In [86]:
MSTr = SSTr/(dof1)
MSE = SSE/(dof2)
F_o = MSTr/MSE
F_o

2.031569173630456

In [87]:
p_val = 1 - f.cdf(F_o,dof1,dof2)
p_val
#Since the p-value is greater than 0.05,  we are unable to reject H_o and thus conclude that when alpha = 0.05 we are not
#able to prove in a statistically significant way that the population means from the different locations are different

0.21195976093738766

In [89]:
#Part B: Tukey Significant Difference Procedure
#First, q_alpha is found from the textbook table:
dof1 = df.shape[1]
dof2 = df.shape[0]*df.shape[1] - df.shape[1]
dof1, dof2 #These are used in the table

(3, 6)

In [118]:
#Since we are now comparing if the collection periods differ, we need to create a new dataframe with the data in diff way
df = pd.DataFrame()
df['7 am'] = [50,80,62]
df['Noon'] = [45, 52, 48]
df['6 pm'] = [57, 74, 68]
q_alp = 5.153
df_mean = df.mean()
df_std = df.std()
SSE = ((df.count()-1)*df_std**2).sum()
MSE = SSE/(3*3-3)
MSE

104.8888888888889

In [119]:
#Next, we do pairwise distances
from sklearn.metrics import pairwise_distances 
distances = pairwise_distances(df_mean.values.reshape(-1,1))
distances

array([[ 0.        , 15.66666667,  2.33333333],
       [15.66666667,  0.        , 18.        ],
       [ 2.33333333, 18.        ,  0.        ]])

In [122]:
#We can then calculate T, which uses MSE from the ANOVA in part A
T = q_alp * (MSE/df.shape[0]) ** 0.5
T

30.469424927415474

In [123]:
#Then, we loop through all the pairs and print out when a pairwise distance is significant (greater than T):
for i in np.arange(df.shape[1]):
    for j in np.arange(df.shape[1]):
        if np.abs(df_mean[i] - df_mean[j]) > T:
            print(f'The mean value of {i} is different from {j} and the difference is {np.abs(df_mean[i] - df_mean[j])}')

In [124]:
#Since no outputs are printed, the Tukey's multiple comparison procedure determines that we cannot determine
#a statistically significant difference between the means of the samples at the different times

In [125]:
df

,7 am,Noon,6 pm
0,50,45,57
1,80,52,74
2,62,48,68
